In [1]:
import numpy as np
import torch

# Training Data

In [3]:
# Input: Temperature, Rainfall and Humidity

inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype = 'float32')

In [4]:
#Target variables (Apples, Oranges):

targets = np.array([[56, 70],
                   [81, 101],
                   [119, 133],
                   [22, 37],
                   [103, 119]], dtype='float32')


In [5]:
#Convert inputs and targets to tensors:
inputs = torch.from_numpy(inputs)

targets = torch.from_numpy(targets)

print(targets)
print(inputs)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])
tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])


# Linear Regression from Scratch:

In [6]:
#Weights and biases:
wt = torch.randn(2, 3, requires_grad=True)

bi = torch.randn(2, requires_grad=True)

print(wt)
print(bi)

tensor([[-0.2928, -1.5792, -0.5304],
        [ 2.5737,  1.2439,  1.8113]], requires_grad=True)
tensor([0.2256, 3.3572], requires_grad=True)


In [11]:
def model(x):
    return x@wt.t() + bi

In [12]:
#Generate Predictions:
pred = model(inputs)
print(pred)

tensor([[-149.7638,  352.4663],
        [-199.3361,  462.9525],
        [-267.6249,  499.0085],
        [-117.1730,  386.3837],
        [-208.7097,  427.1490]], grad_fn=<AddBackward0>)


In [13]:
#Compare the targets:
print (targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [15]:
diff = pred - targets
torch.sum(diff*diff)/diff.numel()

tensor(94872.1719, grad_fn=<DivBackward0>)

In [16]:
#MSE Loss:
def mse(t1, t2):
    diff = t1 -t2
    return torch.sum(diff*diff)/diff.numel()

In [17]:
#Compute loss:
loss = mse(pred, targets)
print(loss)

tensor(94872.1719, grad_fn=<DivBackward0>)



# Compute Gradients:

In [18]:
#Compute Gradients:
loss.backward()

In [20]:
print(wt)
print(wt.grad)

tensor([[-0.2928, -1.5792, -0.5304],
        [ 2.5737,  1.2439,  1.8113]], requires_grad=True)
tensor([[-21974.2617, -25234.4082, -15236.5332],
        [ 28459.9746,  28885.6016,  18207.4258]])


In [22]:
#Settings weights and biases to zero:
wt.grad.zero_()
bi.grad.zero_()
print(wt.grad)
print(bi.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


# Adjust weights and biases by using Gradient Descent Approach:

In [23]:
#1. Generate the predictions from the model:
pred = model(inputs)
print(pred)

tensor([[-149.7638,  352.4663],
        [-199.3361,  462.9525],
        [-267.6249,  499.0085],
        [-117.1730,  386.3837],
        [-208.7097,  427.1490]], grad_fn=<AddBackward0>)


In [24]:
#2. Calculate the loss:
loss = mse(pred, targets)
print(loss)

tensor(94872.1719, grad_fn=<DivBackward0>)


In [25]:
#3. Compute the gradient:
loss.backward()
print(wt.grad)
print(bi.grad)

tensor([[-21974.2617, -25234.4082, -15236.5332],
        [ 28459.9746,  28885.6016,  18207.4258]])
tensor([-264.7215,  333.5920])


In [26]:
#4. Adjust weights and reset gradients:
with torch.no_grad():
    wt -= wt.grad * 1e-5
    bi -= bi.grad * 1e-5
    wt.grad.zero_()
    bi.grad.zero_()

In [27]:
print(wt)
print(bi)

tensor([[-0.0731, -1.3268, -0.3780],
        [ 2.2891,  0.9550,  1.6292]], requires_grad=True)
tensor([0.2283, 3.3539], requires_grad=True)


In [28]:
pred = model(inputs)
loss = mse(pred, targets)
print(loss)

tensor(64571.5859, grad_fn=<DivBackward0>)


In [35]:
#Train for 100 epochs:
for i in range(100):
    pred = model(inputs)
    loss = mse(pred, targets)
    loss.backward()
    with torch.no_grad():
        wt -=wt.grad * 1e-5
        bi -=bi.grad * 1e-5
        wt.grad.zero_()
        bi.grad.zero_()
        

In [36]:
#Again calculate the loss:
pred = model(inputs)
loss = mse(pred, targets)
print(loss)

tensor(14.6036, grad_fn=<DivBackward0>)


In [37]:
#Predictions:
print(pred)

tensor([[ 58.4351,  71.9630],
        [ 81.8040, 101.4033],
        [117.5685, 128.6426],
        [ 28.1102,  43.3527],
        [ 97.1635, 117.3715]], grad_fn=<AddBackward0>)


In [38]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])



# Linear Regression Using Pytorch builtin Functions:

In [40]:
import torch.nn as nn

In [42]:
# Input: Temperature, Rainfall and Humidity

inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58],
                   [102, 43, 37], [69, 96, 70], [73, 67, 43],
                  [91, 88, 64], [87, 134, 58], [102, 43, 47],
                  [69, 96, 70], [73, 67, 43], [91, 88, 64],
                  [87, 134, 58], [102, 43, 47], [69, 96, 70]], dtype = 'float32')

#Targets: Apples and Oranges:

targets = np.array([[56, 70],[81, 101],[119, 133],
                   [22, 37], [103, 119], [56, 70],
                   [81, 101],[119, 133], [22, 37],
                   [103, 119], [56, 70], [81, 101],
                   [119, 133], [22, 37],[103, 119]], dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [43]:
inputs


tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  47.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  47.],
        [ 69.,  96.,  70.]])

# Dataset and Data Loader:

In [44]:
from torch.utils.data import TensorDataset

In [45]:
#Define Dataset:

train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [46]:
from torch.utils.data import DataLoader

In [47]:
#Define Data Loader:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [48]:
for xb, yb in train_dl:
    print('batch:')
    print(xb)
    print(yb)

batch:
tensor([[ 69.,  96.,  70.],
        [ 91.,  88.,  64.],
        [102.,  43.,  47.],
        [ 91.,  88.,  64.],
        [ 73.,  67.,  43.]])
tensor([[103., 119.],
        [ 81., 101.],
        [ 22.,  37.],
        [ 81., 101.],
        [ 56.,  70.]])
batch:
tensor([[ 69.,  96.,  70.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [ 87., 134.,  58.]])
tensor([[103., 119.],
        [ 81., 101.],
        [119., 133.],
        [ 56.,  70.],
        [119., 133.]])
batch:
tensor([[ 87., 134.,  58.],
        [102.,  43.,  47.],
        [ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.]])
tensor([[119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 22.,  37.],
        [ 56.,  70.]])


In [49]:
#Define Model:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)


Parameter containing:
tensor([[-0.4887,  0.4568, -0.0205],
        [-0.2202,  0.2933,  0.5738]], requires_grad=True)
Parameter containing:
tensor([-0.4486, -0.1148], requires_grad=True)


In [51]:
#Parameters:
list(model.parameters())

[Parameter containing:
 tensor([[-0.4887,  0.4568, -0.0205],
         [-0.2202,  0.2933,  0.5738]], requires_grad=True),
 Parameter containing:
 tensor([-0.4486, -0.1148], requires_grad=True)]

In [52]:
#Generate Predictions:
pred = model(inputs)
pred

tensor([[ -6.3938,  28.1338],
        [ -6.0261,  42.3786],
        [ 17.0646,  53.3090],
        [-31.4057,  11.2659],
        [  8.2562,  53.0121],
        [ -6.3938,  28.1338],
        [ -6.0261,  42.3786],
        [ 17.0646,  53.3090],
        [-31.6102,  17.0036],
        [  8.2562,  53.0121],
        [ -6.3938,  28.1338],
        [ -6.0261,  42.3786],
        [ 17.0646,  53.3090],
        [-31.6102,  17.0036],
        [  8.2562,  53.0121]], grad_fn=<AddmmBackward0>)

# Loss Function:

In [53]:
#Import nn.Functional
import torch.nn.functional as F

In [54]:
#Define loss function:
loss_fn = F.mse_loss

In [58]:
loss = loss_fn(pred, targets)
print(loss)

tensor(5008.2104, grad_fn=<MseLossBackward0>)


# Optimizer (Stochastic Gradient Descent SDG):

In [59]:
#Define optimizer:

opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

# Train the model:

In [66]:
#Utility function to train the model:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    #Repeat for given number of epochs:
    for epoch in range(num_epochs):
        
        #Train with batches of data:
        for xb, yb in train_dl:
            
            #Generate Predictions:
            pred = model(xb)
            
            #Calculate loss:
            loss = loss_fn(pred, yb)
            
            #Compute Gradients:
            loss.backward()
            
            #Update Parameters using Gradient:
            opt.step()
            
            #Reset the gradient to zero:
            opt.zero_grad()
            
        #Print the progress:
        if (epoch + 1) % 10 == 0:
            print('Epoch [{}/{}], loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    

In [67]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], loss: 18.4189
Epoch [20/100], loss: 49.5084
Epoch [30/100], loss: 22.6867
Epoch [40/100], loss: 19.7489
Epoch [50/100], loss: 28.0836
Epoch [60/100], loss: 22.8048
Epoch [70/100], loss: 11.5268
Epoch [80/100], loss: 13.5247
Epoch [90/100], loss: 13.5487
Epoch [100/100], loss: 12.9098


In [68]:
#Generate Predictions:
pred = model(inputs)

In [69]:
pred

tensor([[ 57.0275,  69.4848],
        [ 79.4235,  98.7270],
        [123.5042, 134.7186],
        [ 21.8236,  35.3580],
        [ 96.6833, 116.9905],
        [ 57.0275,  69.4848],
        [ 79.4235,  98.7270],
        [123.5042, 134.7186],
        [ 25.5208,  43.2746],
        [ 96.6833, 116.9905],
        [ 57.0275,  69.4848],
        [ 79.4235,  98.7270],
        [123.5042, 134.7186],
        [ 25.5208,  43.2746],
        [ 96.6833, 116.9905]], grad_fn=<AddmmBackward0>)

In [70]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])